In [1]:
import cadquery as cq
import jupyter_cadquery

jupyter_cadquery.set_defaults(ortho=False)

Overwriting auto display for cadquery Workplane and Shape


In [2]:
from dataclasses import dataclass
import math


@dataclass
class HeadParams:
    circumference: float

    @property
    def diameter(self):
        return self.circumference / math.pi

    @property
    def radius(self):
        return self.diameter / 2


@dataclass
class HeadbandParams:
    # The thickness of the headband
    thickness: float
    # The height of the headband
    height: float
    # The length the headband extends past the arc
    # As this increases it takes on a horseshoe shape
    extension_length: float
    # The radius of the attachment loops
    attachment_radius: float
    # The thickness of the attachment loops
    attachment_thickness: float
    # Positions of  Kemo tweeters (degrees relative to 0˚ = front)
    tweeter_positions: list[float]
    # How far forward to push the tweeters in the headband
    tweeter_indent: float
    # Microphone clip height
    mic_clip_height: float
    # Microphone clip clearance (~width of mic board)
    mic_clip_clearance: float
    # Microphone clip protrusion from side
    mic_clip_protrusion: float
    # Microphone clip thickness
    mic_clip_thickness: float


@dataclass
class HeadboxParams:
    # The depth of the headbox
    depth: float
    # The thickness of the control board holder
    wall_thickness: float
    # The peneration of the head into the headbox
    head_penetration: float
    # The cutout cylinder penetration
    deep_cutout_penetration: float
    # The radius of the cutout cylinder
    deep_cutout_radius: float
    # The strap cutout width
    strap_cutout_width: float
    # The strap cutout height
    strap_cutout_height: float
    # The strap cutout depth
    strap_cutout_depth: float


@dataclass
class ControlBoardParams:
    # The width of the control board
    width: float
    # The height of the control board
    height: float
    # The depth of the control board
    depth: float


@dataclass
class BatteryParams:
    # The width of the battery
    width: float
    # The height of the battery
    height: float
    # The depth of the battery
    depth: float


@dataclass
class HeadsetParams:
    front_head: HeadParams
    back_head: HeadParams
    headband: HeadbandParams
    headbox: HeadboxParams
    battery: BatteryParams
    control_board: ControlBoardParams


In [3]:
front_head = HeadParams(circumference=470)
back_head = HeadParams(circumference=560)

headband = HeadbandParams(
    thickness=6,
    height=45,
    extension_length=20,
    attachment_radius=10,
    attachment_thickness=3,
    tweeter_positions=[-60, 0, 60],
    tweeter_indent=1,
    mic_clip_height=6,
    mic_clip_protrusion=10,
    mic_clip_clearance=2.75,
    mic_clip_thickness=2,
)

headbox = HeadboxParams(
    depth=8,
    wall_thickness=2,
    head_penetration=20,
    deep_cutout_penetration=7,
    deep_cutout_radius=7,
    strap_cutout_depth=7,
    strap_cutout_height=15,
    strap_cutout_width=5,
)

battery = BatteryParams(
    width=62,
    height=39,
    depth=9,
)

control_board = ControlBoardParams(
    width=90,
    height=64,
    depth=18,
)

headset = HeadsetParams(
    front_head=front_head,
    back_head=back_head,
    headband=headband,
    headbox=headbox,
    battery=battery,
    control_board=control_board,
)

In [4]:
def create_headbox_arc(
    width: float,
    height: float,
    head_radius: float,
    head_radius_penetration: float,
):
    arc_box = cq.Workplane("XZ").box(
        width,
        height,
        head_radius_penetration,
        centered=(True, True, False),
    )
    arc_cyclinder = (
        cq.Workplane("XY", origin=(0, head_radius - head_radius_penetration))
        .circle(head_radius)
        .extrude(height, both=True)
    )
    y_depth = arc_box.faces("<Y").val().Center().y
    return arc_box.cut(arc_cyclinder).translate((0, -y_depth))


def create_cutout_cylinder(
    radius: float,
    height: float,
    penetration: float,
):
    return (
        cq.Workplane("XY", origin=(0, radius - penetration))
        .circle(radius)
        .extrude(height / 2, both=True)
    )


def create_battery_holder(
    battery_width: float,
    battery_height: float,
    battery_depth: float,
    container_width: float,
    container_height: float,
):
    outer_box = cq.Workplane("XZ").box(
        container_width,
        container_height,
        battery_depth,
        centered=(True, True, False),
    )
    inner_box = cq.Workplane("XZ").box(
        battery_width,
        battery_height,
        battery_depth,
        centered=(True, True, False),
    )
    return outer_box.cut(inner_box)


def create_control_board_holder(
    control_board_width: float,
    control_board_height: float,
    control_board_depth: float,
    container_width: float,
    container_height: float,
    wall_thickness: float,
    ethernet_cutout_width: float = 20,
):
    outer_box = cq.Workplane("XZ").box(
        container_width,
        container_height,
        control_board_depth + wall_thickness,
        centered=(True, True, False),
    )

    inner_box = (
        cq.Workplane("XZ")
        .box(
            control_board_width,
            control_board_height + 2 * wall_thickness,
            control_board_depth,
            centered=(True, True, False),
        )
        .translate((0, 0, wall_thickness))
    )
    ethernet_x = (control_board_width - ethernet_cutout_width) / 2
    ethernet_z = -(control_board_height - ethernet_cutout_width) / 2 - wall_thickness
    ethernet_cutout = (
        cq.Workplane("XZ")
        .box(
            ethernet_cutout_width,
            ethernet_cutout_width,
            control_board_depth,
            centered=(True, True, False),
        )
        .translate((ethernet_x, 0, ethernet_z))
    )
    return outer_box.cut(inner_box).cut(ethernet_cutout)


def bind_to_top_surface(box: cq.Workplane, target: cq.Workplane, cut: bool = False):
    box_y_extent = box.faces("<Y").val().Center().y
    target_y_extent = target.faces("<Y" if cut else ">Y").val().Center().y
    target = target.translate((0, box_y_extent - target_y_extent, 0))
    if cut:
        return box.cut(target)
    else:
        return box.union(target)


def create_headbox(hs: HeadsetParams):
    headbox_height = 2 * hs.headbox.wall_thickness + hs.control_board.height
    headbox_width = 2 * hs.headbox.wall_thickness + hs.control_board.width
    arc = create_headbox_arc(
        width=headbox_width,
        height=headbox_height,
        head_radius=hs.back_head.radius,
        head_radius_penetration=hs.headbox.head_penetration,
    )
    box = cq.Workplane("XZ").box(
        headbox_width,
        headbox_height,
        hs.headbox.depth,
        centered=(True, True, False),
    )
    cutout_cylinder = create_cutout_cylinder(
        radius=hs.headbox.deep_cutout_radius,
        height=3 * hs.headbox.strap_cutout_height,
        penetration=hs.headbox.deep_cutout_penetration,
    )
    box = box.union(arc).cut(cutout_cylinder)
    box = bind_to_top_surface(
        box=box,
        target=cq.Workplane("XZ").box(
            headbox_width,
            hs.headbox.strap_cutout_height,
            hs.headbox.strap_cutout_depth,
            centered=(True, True, True),
        ),
        cut=True,
    )
    box = bind_to_top_surface(
        box=box,
        target=create_battery_holder(
            battery_width=hs.battery.width,
            battery_height=hs.battery.height,
            battery_depth=hs.battery.depth,
            container_width=headbox_width,
            container_height=headbox_height,
        ),
    )
    box = bind_to_top_surface(
        box=box,
        target=create_control_board_holder(
            control_board_width=hs.control_board.width,
            control_board_height=hs.control_board.height,
            control_board_depth=hs.control_board.depth,
            container_width=headbox_width,
            container_height=headbox_height,
            wall_thickness=hs.headbox.wall_thickness,
        ),
    )
    return box


create_headbox(headset)

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…